# Обучение FastText

В бейзлайне предполагается использовать векторы из библиотеки FastText, которые в данном ноутбуке обучатся на домен чеков

In [2]:
from gensim.models.fasttext import FastText
import pandas as pd
import numpy as np

Выгрузим все чеки из размеченного датасета для обучения, а также из тестового датасета

In [3]:
train_df = pd.read_csv("data/train_supervised_dataset.csv").fillna("")
uns_train_df = pd.read_csv("data/train_unsupervised_dataset.csv").fillna("")
test_df = pd.read_csv("data/test_dataset.csv")
names = pd.concat((train_df[["name"]], test_df, uns_train_df[["name"]])).reset_index(drop=True)
names

,name,id
0,Petmax Бантик леопард с красн розой 2шт,NaN
1,87191 Бусы для елки шарики_87191,NaN
2,Футболка Piazza Italia WR011446881,NaN
3,7) YI572-03X-ONE ЗАКОЛКА ДЛЯ ВОЛОС ДЛЯ ДЕВОЧКИ,NaN
4,Одежда (вес) 1500,NaN
...,...,...
1029995,F-2296 Спонж д/макияжа фигурный (шт),NaN
1029996,4 5702737510597 69.88 Дифф/Arom/1601,NaN
1029997,Матрас надувной 540*74см Tropical Bird запл.д/...,NaN
1029998,"пододеяльник СТМ Страйп 3-сп, размер: 796х185с...",NaN


In [4]:
train_df['good'].value_counts(1)

               0.03276
брюки          0.01356
пиво           0.01308
вода           0.01232
печенье        0.01164
                ...   
молитвослов    0.00004
медведь        0.00004
супница        0.00004
овчина         0.00004
поддон         0.00004
Name: good, Length: 2820, dtype: float64

In [5]:
train_df['brand'].value_counts(1)

                        0.34020
яшкино                  0.00480
агуша                   0.00364
коровка из кореновки    0.00272
махеевъ                 0.00252
                         ...   
пласти лаб              0.00004
феррогематоген          0.00004
cbr                     0.00004
helcom                  0.00004
smail                   0.00004
Name: brand, Length: 6976, dtype: float64

In [6]:
from natasha import (
    Segmenter,
    MorphVocab,
    
    NewsEmbedding,
    NewsMorphTagger,
    NewsSyntaxParser,
    NewsNERTagger,
    
    PER,
    NamesExtractor,

    Doc
)

In [7]:
segmenter = Segmenter()
morph_vocab = MorphVocab()
emb = NewsEmbedding()
morph_tagger = NewsMorphTagger(emb)
syntax_parser = NewsSyntaxParser(emb)

names_extractor = NamesExtractor(morph_vocab)

In [8]:
def lemmatize(words):
    text = ' '.join(words)
    
    doc = Doc(text)
    doc.segment(segmenter)
    doc.tag_morph(morph_tagger)
    for token in doc.tokens:
        token.lemmatize(morph_vocab)
    
    words = []
    for token in doc.tokens:
        words.append(token.lemma)
    return words

In [28]:
import re

from cyrtranslit import to_cyrillic

def preprocess_text(text):
    
    text = re.sub('\d+', '0', text)  # replace numbers to 0/''
    text = re.sub(r'(?<=[a-zа-я])([A-ZА-Я])', r' \1', text) # сплитим по заглав. букв
    text = re.sub('д/', 'для ', text)
    text = re.sub(r'\s+', ' ', text)  # remove extra spaces
    text = re.sub(r'[^\w\s]', ' ', text)  # remove punctuation
    words = []
    for w in text.lower().split():
        if bool(re.search('[a-z]', w)) and bool(re.search('[а-я]', w)):
            words.append(to_cyrillic(w))
        else:
            words.append(w)
#     words = lemmatize(words)
    return words

In [29]:
text = names['name'].sample().values[0]
text

'ID180886 Бассейн Intex'

In [30]:
%%timeit
preprocess_text(text)

11.8 µs ± 486 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [31]:
preprocess_text(text)

['id0', 'бассейн', 'intex']

In [32]:
%%time
names["tokens"] = names["name"].apply(preprocess_text)
names

CPU times: user 22 s, sys: 648 ms, total: 22.6 s
Wall time: 22.6 s


,name,id,tokens
0,Petmax Бантик леопард с красн розой 2шт,NaN,"[petmax, бантик, леопард, с, красн, розой, 0шт]"
1,87191 Бусы для елки шарики_87191,NaN,"[0, бусы, для, елки, шарики_0]"
2,Футболка Piazza Italia WR011446881,NaN,"[футболка, piazza, italia, wr0]"
3,7) YI572-03X-ONE ЗАКОЛКА ДЛЯ ВОЛОС ДЛЯ ДЕВОЧКИ,NaN,"[0, yi0, 0x, one, заколка, для, волос, для, де..."
4,Одежда (вес) 1500,NaN,"[одежда, вес, 0]"
...,...,...,...
1029995,F-2296 Спонж д/макияжа фигурный (шт),NaN,"[f, 0, спонж, для, макияжа, фигурный, шт]"
1029996,4 5702737510597 69.88 Дифф/Arom/1601,NaN,"[0, 0, 0, 0, дифф, arom, 0]"
1029997,Матрас надувной 540*74см Tropical Bird запл.д/...,NaN,"[матрас, надувной, 0, 0см, tropical, bird, зап..."
1029998,"пододеяльник СТМ Страйп 3-сп, размер: 796х185с...",NaN,"[пододеяльник, стм, страйп, 0, сп, размер, 0х0..."


In [27]:
# names.to_csv('data/natasha_preprocc_names.csv', index=False)

Обученные векторы положим в файл `fasttext.model` для использования в моделях

In [33]:
%%time
FastText(names["tokens"], vector_size=128, window=5, min_count=5).save("fasttext_models/fasttext_128_cap_split.model")

CPU times: user 3min 27s, sys: 4.06 s, total: 3min 31s
Wall time: 1min 19s
